In [1]:
%load_ext autoreload   

In [2]:
k = 1000
model = 'gpt-4o-mini'

In [3]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from meeting_search import MeetingSearch, build_context_string
from typing import List

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens

from vexa import VexaAPI
from core import system_msg, user_msg,generic_call_stream
from datetime import datetime

from prompts import Prompts
prompts = Prompts()


from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens, assistant_msg
from pydantic_models import QueryPlan, Query


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
vexa = VexaAPI()
analyzer = MeetingSearch()
print(analyzer.count_documents())


User information retrieved successfully.
49


In [5]:
vexa.user_name


'Dmitriy Grankin'

In [6]:
vexa.user_id

'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'

In [10]:
async def generate_output(query, messages, user_id=None, user_name=None):
    query_ = ' '.join([m.content for m in messages]) + ' ' + query
    queries = await analyzer.generate_search_queries(query_,user_id=user_id,user_name=user_name)
    print(queries)
    
    if not user_id and not user_name:
        raise ValueError("Either user_id or user_name must be provided")
    
    summaries = analyzer.get_summaries(user_id=user_id, user_name=user_name)
    full_context, meeting_ids = await analyzer.build_context(queries, summaries, include_all_summaries=False, user_id=user_id, user_name=user_name)
    
    pref = "Based on the following context, answer the question:" if len(messages) == 0 else "Follow-up request:"
    user_info = f"The User is {user_name}"
    messages_context = [
        system_msg(prompts.perplexity + f'. {user_info}'), 
        user_msg(f"Context:\n{full_context}")
    ] + messages + [user_msg(f"{pref} {query}")]
    
    output = await generic_call_stream(messages_context)
    messages.append(user_msg(query))
    messages.append(assistant_msg(output))
    return output, messages, meeting_ids, full_context

In [70]:
async def generate_output(query, messages, user_id=None, user_name=None, thread_id=None):
    query_ = ' '.join([m.content for m in messages]) + ' ' + query
    queries = await analyzer.generate_search_queries(query_, user_id=user_id, user_name=user_name)
    print(queries)
    
    if not user_id and not user_name:
        raise ValueError("Either user_id or user_name must be provided")
    
    summaries = analyzer.get_summaries(user_id=user_id, user_name=user_name)
    full_context, meeting_ids = await analyzer.build_context(queries, summaries, include_all_summaries=False, user_id=user_id, user_name=user_name)
    
    pref = "Based on the following context, answer the question:" if len(messages) == 0 else "Follow-up request:"
    user_info = f"The User is {user_name}"
    messages_context = [
        system_msg(prompts.perplexity + f'. {user_info}'), 
        user_msg(f"Context:\n{full_context}")
    ] + messages + [user_msg(f"{pref} {query}")]
    
    output = await generic_call_stream(messages_context)
    messages.append(user_msg(query))
    messages.append(assistant_msg(output))

    # Create or update thread
    thread_manager = ThreadManager()
    if not thread_id:  # New conversation
        messages_str = ';'.join([m.content for m in messages if m.role == 'user'])
        thread_name = await ThreadName.call([user_msg(messages_str)])
        thread_id = thread_manager.create_thread(user_id=user_id, thread_name=thread_name.thread_name, messages=messages)
    else:  # Existing conversation
        thread_manager.update_thread(thread_id, messages)

    return output, messages, meeting_ids, full_context, thread_id

In [71]:
messages=[]

In [72]:
from pydantic_models import ThreadName

In [73]:
messages_str = ';'.join([m.content for m in messages if m.role == 'user'])

In [67]:
await ThreadName.call([user_msg(messages_str)])


(ThreadName(thread_name='who do i work with?'),
 ChatCompletion(id='chatcmpl-AJMnZ5Rgbyhq4Bl3vINXacc3iehsE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lPQBPOq64w5LdngpZEEGV5wb', function=Function(arguments='{"thread_name":"who do i work with?"}', name='ThreadName'), type='function')]))], created=1729179233, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e2bde53e6e', usage=CompletionUsage(completion_tokens=11, prompt_tokens=101, total_tokens=112, completion_tokens_details=None, prompt_tokens_details=None)))

In [24]:
query = 'who do i work with?'
output, messages,meeting_ids,full_context = await generate_output(query, messages,user_id=vexa.user_id,user_name=vexa.user_name)

[{'start': None, 'end': None, 'vector_search_query': 'Dmitriy Grankin'}, {'start': None, 'end': None, 'vector_search_query': 'Olga Nemirovskaya'}, {'start': None, 'end': None, 'vector_search_query': 'Tatiana Sukhova'}, {'start': None, 'end': None, 'vector_search_query': 'Rick Tousseyn'}, {'start': None, 'end': None, 'vector_search_query': 'Nikolay'}, {'start': None, 'end': None, 'vector_search_query': 'Umar Lateef'}, {'start': None, 'end': None, 'vector_search_query': 'Sergey Ryabenko'}, {'start': None, 'end': None, 'vector_search_query': 'David Sterry'}, {'start': None, 'end': None, 'vector_search_query': 'Angelina Geru'}, {'start': None, 'end': None, 'vector_search_query': 'Igor Bessonov'}]
You work with several individuals, including Umar Lateef, Tatiana Sukhova, Rick Tousseyn, Olga Nemirovskaya, Angelina Geru, and Igor Bessonov. Your discussions cover various topics such as social media strategies, product development, marketing strategies, and technical troubleshooting.


In [36]:
from core import Msg

In [37]:
Msg

core.Msg

In [38]:
from assistant_thread import SearchAssistantThreadManager

In [44]:

thread_manager = SearchAssistantThreadManager()

# Then you can use it as before:
thread_id = thread_manager.create_thread(user_id="user123", thread_name="My First Thread1", messages=messages,thread_id=thread_id)

In [45]:
thread_id

'73ace234-2c88-4d4b-bc2a-24bdbe1691fc'

In [46]:
thread_manager.get_thread(thread_id)

SearchAssistantThread(thread_id='73ace234-2c88-4d4b-bc2a-24bdbe1691fc', user_id='user123', thread_name='My First Thread1', messages=[Msg(role='user', content='who do i work with?', stage=None), Msg(role='assistant', content='You work with several individuals, including Umar Lateef, Tatiana Sukhova, Rick Tousseyn, Olga Nemirovskaya, Angelina Geru, and Igor Bessonov. Your discussions cover various topics such as social media strategies, product development, marketing strategies, and technical troubleshooting.', stage=None)], timestamp=datetime.datetime(2024, 10, 17, 15, 20, 46, 572578))

In [21]:
messages

[Msg(role='user', content='who is the user?', stage=None),
 Msg(role='assistant', content='The user is Dmitriy Grankin.', stage=None),
 Msg(role='user', content='who do i work with?', stage=None),
 Msg(role='assistant', content='You work with Sergey Ryabenko, as indicated in the discussions regarding the Figma plugin for HTML generation and other technical meetings.', stage=None),
 Msg(role='user', content='who do i work with?', stage=None),
 Msg(role='assistant', content='You work with Sergey Ryabenko, as well as other team members involved in various discussions and testing sessions.', stage=None)]

In [16]:
print(full_context)

# [1] Figma Plugin Discussion (2024-10-14 13:21):
In a technical meeting on October 14, 2024, Sergey Ryabenko and Dmitriy Grankin discussed the use of a Figma plugin for HTML generation. They addressed issues with the plugin's performance, the need for responsive design, and the potential use of Tailwind CSS for styling. The conversation included troubleshooting steps and the importance of exporting code effectively.
## Related Quotes:
:  Все, Дим, давай. :  Нет, попробуем, попробуем сперва, попробуем. :  Давай, пока.
Dmitriy Grankin:  export code get pro давай export Sergey Ryabenko:  download zip Dmitriy Grankin:  Не, окей, а посмотреть-то можно, что получилось? Dmitriy Grankin:  Ну вот ты смотришь, видимо. Dmitriy Grankin:  Так я же хочу посмотреть на код. Sergey Ryabenko:  Ну, сделай инспект. Sergey Ryabenko:  А-а-а. Dmitriy Grankin:  Логично. Sergey Ryabenko:  Ты можешь, знаешь, что попробовать? Sergey Ryabenko:  Подожди, кликни правой кнопкой, там должно быть... Sergey Ryabenko: 

In [152]:
count_tokens(full_context)

8835

In [153]:
30000*100/1000000*0.15

0.44999999999999996

In [149]:
query = 'list all condidate names considered and your assessment'
output, messages,meeting_ids,full_context = await generate_output(query, messages)

[{'start': None, 'end': None, 'vector_search_query': 'candidate evaluation for backend positions, concerns about qualifications and fit, ongoing evaluation process, overqualified candidates, coding abilities assessment'}, {'start': None, 'end': None, 'vector_search_query': 'status of backend position, open positions, candidate qualifications, technical role evaluations, long-term fit for team'}, {'start': None, 'end': None, 'vector_search_query': 'names of candidates considered for backend roles, assessment of coding skills, theoretical knowledge vs practical abilities, ongoing evaluations'}, {'start': None, 'end': None, 'vector_search_query': 'discussions on backend candidate evaluations, concerns about overqualified candidates, importance of practical coding skills, team fit'}, {'start': None, 'end': None, 'vector_search_query': 'current status of backend position, candidate evaluation process, qualifications of candidates, ongoing search for suitable candidates'}]
The discussions in

In [142]:
query = 'specific names please'
output, messages,meeting_ids = await generate_output(query, messages)

[{'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'candidates for backend development'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'backend developer position status'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'discussion on backend candidates'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'backend developer hiring process'}, {'start': datetime.datetime(2024, 10, 1, 0, 0), 'end': datetime.datetime(2024, 10, 17, 23, 59, 59), 'vector_search_query': 'October 14 meeting on backend candidates'}]
The context does not provide any specific names of candidates considered for the backend position. The discussions mainly revolved around technical challenges and the need for developers w

In [131]:

meeting_id_ref = 4
dashboard_url = f"https://dashboard.vexa.ai/#{meeting_ids[meeting_id_ref-1]}"
dashboard_url


'https://dashboard.vexa.ai/#8ad52667-1a1d-4aa6-ba9a-a2021fe650bc'